# Mid term

You have **80** minutes for this exam.  

Save your exam in the location **mp248/Exams/Midterm.ipynb** on your **GitLab** repository.  You will likely have to create your **Exams** subdirectory.  

Ensure you push your partial work to your repository *early* and *often*.

**Note**: Invigilators will only offer help with **GIT** issues in the final 20 minutes of the exam.  Students that receive help on GIT issues will be deducted **10%** from their exam grade.

* * *

## Problem 1
1. [80% of grade for P1] How much water do you have to store on your 10m high roof-top in order to store enough potential energy, that - if transformed without losses - could run a 50W bulp for 5 hr? Use a package such as `astropy` that provides the capability to calculate with quantities that have units. 
2. [20%] Express your answer as a callable function that takes the power of the light bulp and the duration of how long it should run as well as the hight of the roof top as input. Use the function with the given input values and print the results in the unit kg. 

In [ ]:
from astropy import constants as c
from astropy import units as u

In [ ]:
def water_mass(P = 50 * u.W,t = 5 * u.hr,h = 10 * u.m):
    '''
    Return water mass required to run light bulp
    
    Parameters:
    -----------
    P : float
      power of light bulp in W
    t : float
      time to run bulp in hrs
    h : float
      height of water to be stored in m
      
    Output:
    -------
    m : mass
      mass of water required
    '''
    E = P * t
    m = E/(c.g0*h)
    return m

In [ ]:
water_mass().to('kg')

* * *

## Problem 2
1. [25%] Make a plot of a square with side length 2 centered at the origin. This means the lower left corner is at the coordinate $(x,y)=(-1,-1)$. Use a solid black line. Add to the plot a circle with radius 1 also centered at the origin. Use a red dashed line.  
2. [25%] Place 100 points inside the square that are randomly distributed in x- and y-direction. Use blue dots.
3. [30%] Determine the fraction of randomly placed points inside the circle. Calculate from this fraction the area of the circle using the known area of the square. Repeat the calculation a couple of times. What is your average answer from 3 calculations?
4. [10%] By how much does this average calculated answer differ from `numpy.pi` in %? 
5. [10%] Is this difference a lack of accuracy or a lack of precision? Explain in one sentence.

In [ ]:
%pylab ipympl

In [ ]:
close(11);figure(11,figsize=(6,6))

x=[-1,1,1,-1,-1]
y=[-1,-1,1,1,-1]
plot(x,y,'k--')

yy= lambda x: sqrt(1 - x**2)
xx = linspace(-1,1,100)
plot(xx,yy(xx),'r-')
plot(xx,-yy(xx),'r-')

In [ ]:
n=100
x=2*random.rand(n)-1
y=2*random.rand(n)-1
plot(x,y,'o')

In [ ]:
area=0
for i in range(0,3):
    n=100
    x=2*random.rand(n)-1
    y=2*random.rand(n)-1
    r = sqrt(x**2 + y**2)
    fraction_circle = len(where(r<=1.)[0])/n
    area += fraction_circle*4.
print(area/(i+1))
print("Fraction %5.2f"%(100*(area/(i+1)-pi)/pi))

This difference is a lack of accuracy. 

Accuracy is a property of the algorithm. It describes how well the calculation **approximates** the exact answer, assuming that in calculating this answer all mathematical operations are carried out with unlimited precision. Precision is the degree to which the discretized equations are reproduced on the computer in the face of round-off errors.


* * *

## Problem 3

In this problem we ask you to write a **simple equation solver** function **modsubdiv(f,x,a,b,eps,k)**

The function **modsubdiv** takes as arguments:

* $f$ a Sympy expression, representing a function of $x$, a Sympy variable.  

* $a$ and $b$ are floats. The equation solver will require that $f(a)$ and $f(b)$ have opposite signs.  We call this the **iteration condition**.  The interval $[a,b]$ we call the **considered interval**.

* **eps** will be a threshold where the function **modsubdiv** stops iteration.  This happens when the length of the **considered interval** is less than or equal to **eps**. 

**modsubdiv** will return three numbers: the endpoints of the final considered interval, and the number of iterations required to meet the threshold condition.

For the iteration itself, we ask you to replace the interval $[a,b]$ with a subinterval of $[a,b]$.  Divide the interval $[a,b]$ into $k$ equally spaced subintervals 

$$[a,b] = [a, a+\frac{b-a}{k}] \cup [a+\frac{b-a}{k}, a+2\frac{b-a}{k}] \cup \cdots \cup [a+(k-1)\frac{b-a}{k}, a+k\frac{b-a}{k}=b].$$ 

Choose the first such subinterval that satisfies the iteration condition. Repeat until the considered interval is no longer than **eps**. 

### Part 3.a [60%]
Write the function **modsubdiv**.  

**Hint:** For a simple check on the correctness of your code, compare your **modsubdiv** output with $k=2$ to the output of the **subdiv** function from Assignment 2.

### Part 3.b [20%]
Compare the number of iterations it takes **modsubdiv** to find the roots of the equation

$$x^2 - 2 = 0$$

with the initial considered interval $[a,b] = [1,2]$ and $eps = \frac{1}{10}, \frac{1}{100}, \frac{1}{1000}, \cdots, \frac{1}{10^6}$, and $k=2, 3, 4, 5$.  Ensure you print the output of **modsubdiv** every time you call it.

### Part 3.c [20%]
Repeat **Part 3.b** for the equation

$$x^x - 2 = 0$$

with initial considered interval $[a,b]=[1,2]$. 

In [ ]:
import numpy as np
import sympy as sp
import operator as op

In [ ]:
## We modify the subdiv function from Assignment 2. 

def modsubdiv(f,x,a,b,eps,k):
    '''
    f is a sympy expression, a function of a single variable, x
    a and b are the initial endpoints
    eps is the stop parameter.  
    k is the number of subintervals to consider, during iteration.
    
    This function repeatedly divides the interval [a,b] into k
    equal-length subintervals.  Finds a subinterval where the function
    f has opposite-signs on the endpoints, then replaces [a,b]
    by that subinterval, until [a,b] is shorter than eps. 
    it then returns the endpoints of the final considered interval, 
    and the iteration count, ct.
    '''
    
    F = sp.lambdify(x, f)
    
    ct = 0
    while (b-a)>eps:
        subints = [[a+i*(b-a)/k, a+(i+1)*(b-a)/k] for i in range(k)]
        for I in subints:
            if F(I[0])*F(I[1])<0.0:
                break
        ct += 1
        a = I[0]
        b = I[1]
        
    return a,b, ct

In [ ]:
## This is the subdiv function from Assignment 2.
def subdiv(f,x,a,b,eps):
    F = sp.lambdify(x, f)
    ct = 0
    while (b-a)>eps:
        mp = (a+b)/2
        ct += 1
        fmp = F(mp)
        fa = F(a)
        fb = F(b)
        if np.sign(fa)==np.sign(fb):
            return("No root.")
        if np.sign(fmp)==np.sign(fa):
            a = mp
            continue
        b = mp
    return a,b, ct

In [ ]:
## (a) Sanity check
x = sp.Symbol('x')
f = x**x - 2

for k in [1,2,3,4,5,6]:
    print("subdiv: ",subdiv(f,x,1.0,2.0, 1/10**k))
    print("modsbd: ",modsubdiv(f, x, 1.0, 2.0, 1/10**k,2))
    print(" ")

In [ ]:
## Part (b)

for k in [2, 3,4,5]:
    print("k ==", k)
    for j in [1,2,3,4,5,6]:
        print("modsbd: ",modsubdiv(f, x, 1.0, 2.0, 1/10**j,k))
        print(" ")

In [ ]:
## Part (c)

f=x**x-2

for k in [2,3,4,5]:
    print("k ==", k)
    for j in [1,2,3,4,5,6]:
        print("modsbd: ",modsubdiv(f, x, 1.0, 2.0, 1/10**j,k))
        print(" ")